In [1]:
from misc import load, load_class_labels
from data_loader import data_loader;
from tensorflow import keras;

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

dl = data_loader(train_info, train_dir, 400, (224, 224));
dl_val = data_loader(val_info, train_dir, 400, (224, 224));


2023-12-28 17:52:05.329694: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 17:52:05.329730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 17:52:05.333295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 17:52:05.349117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 17:52:06.469732: W tensorflow/compiler/tf2

## Visualize augmented samples

In [2]:
index = 0;

In [3]:
from misc import apply_data_augmentation;
import cv2 as cv

images, labels, filenames = dl.get_batch(index);
images_aug = apply_data_augmentation(images);

for image, filename in zip(images_aug, filenames):

    image = cv.cvtColor(image, cv.COLOR_RGB2BGR);
    #cv.imwrite('./dataset/aug_test/' + filename, image)
    
index = index + 1;


## Classification using handcrafted features

In [4]:
from multiprocessing.pool import ThreadPool
from handcrafted import extract_features, color_histograms, sift, gabor_response, lbp
from tqdm import tqdm;
import numpy as np;

sampling = train_info.sample(frac=0.5)
images_names = sampling.iloc[:, 0].values

pool = ThreadPool(processes=14)

angles = np.arange(0, np.pi, np.pi/4)
lambdas = np.arange(0, 1, 0.2);
gammas = [0.5]
max_features = 50

sift_features = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: sift(img, max_features=max_features))]))

gabor = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: gabor_response(img, (10, 10), angles, [5], lambdas, gammas))]))

color = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: color_histograms(img))]))

lbp_features = pool.apply_async(extract_features, (train_dir, images_names,
                                                  [(lambda img: lbp(img))]))



 

sift_features = sift_features.get()
gabor = gabor.get()
color = color.get()
lbp_features = lbp_features.get()

0it [00:00, ?it/s]







4it [00:00, 17.25it/s]



7it [00:00, 18.75it/s]


10it [00:00, 18.70it/s]


12it [00:00, 17.03it/s]




14it [00:00, 16.94it/s]


16it [00:00, 17.23it/s]


19it [00:01, 18.80it/s]


21it [00:01, 18.57it/s]


23it [00:01, 16.73it/s]

25it [00:01, 17.40it/s]


27it [00:01, 17.75it/s]
29it [00:01, 18.27it/s]



32it [00:01, 19.97it/s]


35it [00:01, 21.53it/s]





38it [00:02, 19.97it/s]


41it [00:02, 21.66it/s]


44it [00:02, 19.80it/s]




47it [00:02, 19.39it/s]


50it [00:02, 18.65it/s]


53it [00:02, 20.60it/s]


56it [00:02, 19.06it/s]


58it [00:03, 18.30it/s]


60it [00:03, 18.33it/s]


63it [00:03, 18.52it/s]


67it [00:03, 20.98it/s]


70it [00:03, 22.44it/s]


74it [00:03, 24.72it/s]



78it [00:03, 25.04it/s]





81it [00:04, 21.30it/s]




84it [00:04, 18.69it/s]


88it [00:04, 22.02it/s]


91it [00:04, 22.70it/s]


94it [00:04, 22.12it/s]


98it [00:04, 24.56it/s]




101it [00:04, 23.27it/s]


104it [00:05, 21.24it/s]





107it [00:05, 18.75it/s]


109it [00:05, 17.

In [5]:
from misc import unroll_arrays
import bag_of_words as bow

prob = 1
voc_size = 300

unrolled = unroll_arrays(sift_features, 1);

kmeans = bow.fit(unrolled, vocabulary_size=voc_size, verbose=True, n_init=1, max_iter=500)

Initialization complete
Iteration 0, inertia 238606073856.0.
Iteration 1, inertia 171348426752.0.
Iteration 2, inertia 168282030080.0.
Iteration 3, inertia 166986645504.0.
Iteration 4, inertia 166231212032.0.
Iteration 5, inertia 165709922304.0.
Iteration 6, inertia 165314166784.0.
Iteration 7, inertia 165004935168.0.
Iteration 8, inertia 164757635072.0.
Iteration 9, inertia 164560666624.0.
Iteration 10, inertia 164397318144.0.
Iteration 11, inertia 164262985728.0.
Iteration 12, inertia 164147707904.0.
Iteration 13, inertia 164047306752.0.
Iteration 14, inertia 163963207680.0.
Iteration 15, inertia 163889381376.0.
Iteration 16, inertia 163827466240.0.
Iteration 17, inertia 163772825600.0.
Iteration 18, inertia 163722362880.0.
Iteration 19, inertia 163674718208.0.
Iteration 20, inertia 163632529408.0.
Iteration 21, inertia 163594993664.0.
Iteration 22, inertia 163561799680.0.
Iteration 23, inertia 163530391552.0.
Iteration 24, inertia 163502243840.0.
Iteration 25, inertia 163471753216.0

In [6]:
bow_features = bow.predict(kmeans, sift_features)
print(bow_features.shape)

(47391, 300)


In [7]:
unrolled_color = np.array(color)
unrolled_color = unrolled_color.reshape(unrolled_color.shape[0], -1)
unrolled_gabor = np.array(gabor)
unrolled_lbp = np.array(lbp_features)

all_features = np.concatenate((unrolled_color, bow_features, unrolled_gabor, unrolled_lbp), axis=1)
print(all_features.shape)

(47391, 390)


## Standard classification

### SVM

In [8]:
from sklearn import svm

clf = svm.SVC()
clf.fit(all_features, sampling['label'])

SVC()

## Neural network classification

In [27]:
model = keras.Sequential([
    keras.Input(shape=390),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(251, activation='softmax')
])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [28]:
import pandas as pd
epoch_num = 40;

labels = keras.utils.to_categorical(sampling['label'].values.tolist(), num_classes=251)

for epoch in range(epoch_num):
    #df = df.sample(frac=1);
    model.fit(all_features, labels, batch_size=100, shuffle=True, validation_split=0.1);

  9/427 [..............................] - ETA: 3:41 - loss: 5.5112 - accuracy: 0.0056

In [ ]:
'''
for epoch in range(epoch_num):
    dl.shuffle_dataframe();
    for batch_num in tqdm(range(dl.number_of_batch())):
        (images, labels, _) = dl.get_batch(batch_num)
        features = []
        for image in images:
            image = np.float32(image)
            hist = color_histograms(image);
            features.append(hist)
        
        features = bow.predict(kmeans, features);
        
        labels = keras.utils.to_categorical(labels, num_classes=251)
        model.train_on_batch(features, labels)
'''

In [ ]:
dl_val.shuffle_dataframe()
for batch_num in tqdm(range(dl_val.number_of_batch())):
    (images, labels, _) = dl_val.get_batch(batch_num)
    features = []
    for image in images:
        image = cv.normalize(image, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        image = image[:, :, ::-1];
        #sift_info = sift(image, max_features=max_features)
        features.append(sift_info)
    
    features = bow.predict(kmeans, features);
    
    labels = keras.utils.to_categorical(labels, num_classes=251)
    loss, acc = model.evaluate(features, labels);